In [2]:
import os
os.chdir("../")


In [3]:
os.getcwd()

'/Users/abishekkumaryadav/DataScience/Machine_learning/Customer_segmentation_project'

In [4]:
os.chdir('/Users/abishekkumaryadav/DataScience/Machine_learning/Customer_segmentation_project')

In [23]:
from dataclasses import dataclass
from pathlib import Path

In [29]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data: Path
    test_data: Path
    best_model_path: Path  
    dim_red_model: Path

In [30]:
from src.Customer_segementation.constant import *
from src.Customer_segementation.utils.common import read_yaml, create_directories

In [31]:
class configurationManager: 
    def __init__(self,config_file_path=CONFIG_FILE_PATH,
                 schema_file_path=SCHEMA_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.schema=read_yaml(schema_file_path)
        self.params=read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self)->ModelTrainerConfig:
        config=self.config.model_trainer
        create_directories([config.root_dir])
        
        model_Trainer_config=ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data=config.train_data_scaled_path,
            test_data=config.test_data_scaled_path,
            best_model_path=config.best_model_path,
            dim_red_model=config.dim_red_model_path
            )
        return model_Trainer_config

In [32]:
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
import pandas as pd
from src.Customer_segementation.logger import logger
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, classification_report
from src.Customer_segementation.utils.common import save_object

In [33]:
# pip install dagshub

In [34]:
from sklearn.metrics import silhouette_score
import dagshub
import mlflow
from urllib.parse import urlparse
from src.Customer_segementation.utils.common import load_object

In [43]:
class ModelTainer:
    def __init__(self, config=ModelTrainerConfig) -> None:
        self.config=config

        self.model={
            'kmean': KMeans(n_clusters=4, init='k-means++')
        }

    def model_evaluation(self, x_train, x_test, Model):
        report={}
        dim_red=PCA(n_components=2)
        x_train_scaled=dim_red.fit_transform(x_train)
        x_test_scaled=dim_red.transform(x_test)

        save_object(file_path=Path(self.config.dim_red_model), obj=dim_red)
        
        for mod in range(len(Model)): 
            model= list(Model.values())[mod]

            model= model.fit(x_train_scaled)

            logger.info(f"Labels: {model.labels_}")
            sil_score=silhouette_score(x_train,model.labels_)

            logger.info(f"silhouette_score of kmean: {sil_score}")

           
            report[list(Model.keys())[mod]]=silhouette_score
            # report[list(Model.keys())[mod]]=score
        return report
        
    def inititate_model_trainer(self):
        train_data=self.config.train_data
        test_data=self.config.test_data

        train= pd.read_csv(train_data)
        test=pd.read_csv(test_data)


        # logger.info(f"x_: {train}")
        # logger.info(f"{test}")

        # logger.info(f"Model list :{self.model}")

        report=self.model_evaluation(train, test, self.model)
        logger.info(f"Report of model: {report}")

        # save_object(file_path=Path(self.config.best_model_path), obj=model)
        # save_object(file_path=Path(self.config.dim_red_model), obj=dim_red)
        


In [44]:
try: 
    config_manager=configurationManager()
    model_trainer_config=config_manager.get_model_trainer_config()
    model_trainer=ModelTainer(model_trainer_config)
    model_trainer.inititate_model_trainer()
except Exception as e:
    raise e

[2024-07-01 20:17:47,177 : INFO : common : Yaml file read config/config.yaml successfully]
[2024-07-01 20:17:47,179 : INFO : common : Yaml file read schema.yaml successfully]
[2024-07-01 20:17:47,180 : INFO : common : Yaml file read params.yaml successfully]
[2024-07-01 20:17:47,181 : INFO : common : Directories created ['artifacts']]
[2024-07-01 20:17:47,181 : INFO : common : Directories created ['artifacts/model_trainer']]
[2024-07-01 20:17:47,673 : INFO : common : Object save at: <_io.BufferedWriter name='artifacts/model_trainer/dim_red.h5'>]


/Users/abishekkumaryadav/DataScience/Machine_learning/Customer_segmentation_project/cus_env/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


[2024-07-01 20:17:47,986 : INFO : 2631635086 : Labels: [0 0 0 ... 1 3 2]]
[2024-07-01 20:17:48,737 : INFO : 2631635086 : silhouette_score of kmean: 0.2729232670458257]
